In [ ]:
%matplotlib

import logging
from IPython.display import display, clear_output

import numpy as np
import MDAnalysis as mda
import matplotlib.pyplot as plt

from imdclient.tests.datafiles import NAMD_TOPOL


# %%
logger = logging.getLogger("imdclient.IMDClient")
file_handler = logging.FileHandler("imdclient.log")
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.setLevel(logging.INFO)

timeWindow = 5.0
dtIMD = 0.010
nTimesPlot = (int)(timeWindow/dtIMD)

# %%
u = mda.Universe(NAMD_TOPOL, "imd://localhost:8881", buffer_size = 100*1024*1024)

plt.ion()  # Turn on interactive mode
fig, ax = plt.subplots(figsize=(10, 6))
p1, = ax.plot([], [], label='Nter-Cter')
p2, = ax.plot([], [], label='Glu35-Asp52')
ax.set_xlabel('Time (ps)')
ax.set_ylabel('Distance (Å)')
ax.set_xlim(0, timeWindow)
ax.set_ylim(3, 25)
ax.set_title('Distance vs. Time')
ax.legend()
ax.grid(True)
plt.tight_layout()
display(fig)

min_idx = 0


i = 0
for ts in u.trajectory:
    idx = i % nTimesPlot
    time[idx] = ts.time
    dist1[idx] = np.linalg.norm(Nter.atoms[0].position - Cter.atoms[0].position)
    dist2[idx] = np.linalg.norm(GLU35.atoms[0].position - ASP52.atoms[0].position)

    if i % nTimesPlot == 0:
        min_idx = idx
        ax.set_xlim(time[idx], time[idx] + nTimesPlot*dtIMD)

    if i % timeWindow ==0:
        valid_indices = range(min_idx, idx + 1)
        p1.set_data(time[valid_indices], dist1[valid_indices])
        p2.set_data(time[valid_indices], dist2[valid_indices])
        
        clear_output(wait=True)  # Clear previous output
        display(fig)  # Redisplay the updated figure
    i += 1
# %%
logger.info(f"Parsed {i} frames")
plt.ioff()